In [1]:
import pandas as pd
import numpy as np

In [2]:
from utils import generate_table

In [3]:
missing1_base = pd.read_csv("adult_1group.csv")
missing2_base = pd.read_csv("adult_2groups.csv")

In [4]:
missing1_others = {
    "kmeans (eval on recon)": "adult_1group_kmeans_eval_on_recon.csv",
    "ranking (eval on recon)": "adult_1group_ranking_eval_on_recon.csv",
    "kmeans (supersample)": "adult_1group_kmeans_supersample.csv",
    "ranking (supersample)": "adult_1group_ranking_supersample.csv",
    "kmeans (eval on recon and supersample)": "adult_1group_kmeans_supersample_eval_on_recon.csv",
    "ranking (eval on recon and supersample)": "adult_1group_ranking_supersample_eval_on_recon.csv",
}
missing2_others = {
    "kmeans (eval on recon)": "adult_2groups_kmeans_eval_on_recon.csv",
    "ranking (eval on recon)": "adult_2groups_ranking_eval_on_recon.csv",
    "kmeans (supersample)": "adult_2groups_kmeans_supersample.csv",
    "ranking (supersample)": "adult_2groups_ranking_supersample.csv",
    "kmeans (eval on recon and supersample)": "adult_2groups_kmeans_supersample_eval_on_recon.csv",
    "ranking (eval on recon and supersample)": "adult_2groups_ranking_supersample_eval_on_recon.csv",
}

def normalize_others(file_dict):
    df_all = pd.DataFrame()
    for approach, filename in file_dict.items():
        df = pd.read_csv(filename)
        df.insert(0, "approach", approach)
        df_all = pd.concat([df_all, df], axis="index", ignore_index=True, sort=False)
    return df_all
missing1_others = normalize_others(missing1_others)
missing2_others = normalize_others(missing2_others)

In [5]:
missing1_ens = {
    "true_balancing": "adult_1group_ens_balanced_context.csv",
    "no_balancing": "adult_1group_ens_no_balancing.csv",
    "ranking": "adult_1group_ranking_EnS.csv",
}

def normalize_ens(file_dict):
    df_all = pd.DataFrame()
    for approach, filename in file_dict.items():
        df = pd.read_csv(filename)
        df.insert(0, "approach", approach)
        df["data"].replace({"x_rand_s": "E&S"}, inplace=True)
        df_all = pd.concat([df_all, df], axis="index", ignore_index=True, sort=False)
    return df_all
missing1_ens = normalize_ens(missing1_ens)

In [6]:
dro_missing1 = pd.read_csv("adult_dro_1group.csv")
dro_missing2 = pd.read_csv("adult_dro_2group.csv")

# normalizing this data:
def normalize_dro_results(df):
    df.insert(0, "approach", "no_balancing")
    df.insert(2, "data", "original_data")
    df.insert(4, "wandb_url", "(None)")
    df.insert(5, "cluster_test_acc", float("nan"))
    df.insert(6, "cluster_context_acc", float("nan"))
    df.rename(columns={
        "Seed": "seed",
        "eta": "method",
        "prob_pos_sens_0.0/sens_1.0": "prob_pos_sex_Male_0.0/sex_Male_1.0",
        "TPR_sens_0.0/sens_1.0": "TPR_sex_Male_0.0/sex_Male_1.0",
        "TNR_sens_0.0/sens_1.0": "TNR_sex_Male_0.0/sex_Male_1.0",
    }, inplace=True)
    df["method"] = df["method"].transform(lambda eta: f"DRO (eta={eta})")
    
normalize_dro_results(dro_missing1)
normalize_dro_results(dro_missing2)

In [7]:
missing1 = pd.concat([missing1_base, dro_missing1, missing1_others, missing1_ens], axis="index", ignore_index=True, sort=False)
missing2 = pd.concat([missing2_base, dro_missing2, missing2_others], axis="index", ignore_index=True, sort=False)

In [11]:
m1 = generate_table(
    df=missing1,
    type_renames={"x_rand_s": "ours", "original_data": "baseline"},
    metrics=["Accuracy", "prob_pos_sex_Male_0.0/sex_Male_1.0", "TPR_sex_Male_0.0/sex_Male_1.0", "TNR_sex_Male_0.0/sex_Male_1.0", "cluster_context_acc"],
#     query="~(type == 'baseline' & (approach.str.contains('E&S') | approach.str.contains('kmeans') | approach.str.contains('ranking') | approach == 'true_balancing')) & ~(type == 'ours' & classifier == 'LRCV')",
    query="~(type == 'baseline')"
)
m1

Accuracy  \
type approach                                classifier                                       
E&S  no_balancing                            LRCV                         0.676 $\pm$ 0.029   
                                             Logistic Regression (C=1.0)  0.677 $\pm$ 0.028   
     ranking                                 LRCV                         0.606 $\pm$ 0.008   
                                             Logistic Regression (C=1.0)  0.606 $\pm$ 0.008   
     true_balancing                          LRCV                         0.648 $\pm$ 0.033   
                                             Logistic Regression (C=1.0)  0.651 $\pm$ 0.036   
ours kmeans                                  LRCV                         0.671 $\pm$ 0.025   
                                             Logistic Regression (C=1.0)  0.671 $\pm$ 0.025   
     kmeans (eval on recon and supersample)  LRCV                         0.672 $\pm$ 0.023   
                                             Logistic Regression (C=1.0)  0.673 $\pm$ 0.022   
     kmeans (eval on recon)                  LRCV                         0.640 $\pm$ 0.030   
                                             Logistic Regression (C=1.0)  0.635 $\pm$ 0.028   
     kmeans (supersample)                    LRCV                         0.661 $\pm$ 0.013   
                                             Logistic Regression (C=1.0)  0.661 $\pm$ 0.014   
     no_balancing                            LRCV                         0.666 $\pm$ 0.014   
                                             Logistic Regression (C=1.0)  0.667 $\pm$ 0.015   
     ranking                                 LRCV                         0.679 $\pm$ 0.020   
                                             Logistic Regression (C=1.0)  0.680 $\pm$ 0.021   
     ranking (eval on recon and supersample) LRCV                         0.653 $\pm$ 0.029   
                                             Logistic Regression (C=1.0)  0.655 $\pm$ 0.029   
     ranking (eval on recon)                 LRCV                         0.673 $\pm$ 0.029   
                                             Logistic Regression (C=1.0)  0.675 $\pm$ 0.028   
     ranking (supersample)                   LRCV                         0.678 $\pm$ 0.022   
                                             Logistic Regression (C=1.0)  0.679 $\pm$ 0.025   
     true_balancing                          LRCV                         0.656 $\pm$ 0.044   
                                             Logistic Regression (C=1.0)  0.656 $\pm$ 0.044   

                                                                         prob_pos_sex_Male_0.0/sex_Male_1.0  \
type approach                                classifier                                                       
E&S  no_balancing                            LRCV                                         0.525 $\pm$ 0.262   
                                             Logistic Regression (C=1.0)                  0.525 $\pm$ 0.260   
     ranking                                 LRCV                                         0.298 $\pm$ 0.091   
                                             Logistic Regression (C=1.0)                  0.298 $\pm$ 0.093   
     true_balancing                          LRCV                                         0.504 $\pm$ 0.212   
                                             Logistic Regression (C=1.0)                  0.512 $\pm$ 0.228   
ours kmeans                                  LRCV                                         0.313 $\pm$ 0.153   
                                             Logistic Regression (C=1.0)                  0.313 $\pm$ 0.154   
     kmeans (eval on recon and supersample)  LRCV                                         0.435 $\pm$ 0.221   
                                             Logistic Regression (C=1.0)                  0.440 $\pm$ 0.224   
     kmeans (eval on recon)                  LRCV                                         

In [13]:
m2 = generate_table(
    df=missing2,
    type_renames={"x_rand_s": "ours", "original_data": "baseline"},
    metrics=["Accuracy", "prob_pos_sex_Male_0.0/sex_Male_1.0", "TPR_sex_Male_0.0/sex_Male_1.0", "TNR_sex_Male_0.0/sex_Male_1.0", "cluster_context_acc"],
#     query="~(type == 'baseline' & (approach.str.contains('E&S') | approach.str.contains('kmeans') | approach.str.contains('ranking') | approach == 'true_balancing')) & ~(type == 'ours' & classifier == 'LRCV')",
    query="~(type == 'baseline')"
)
m2

Accuracy  \
type approach                                classifier                                       
ours kmeans                                  LRCV                         0.713 $\pm$ 0.064   
                                             Logistic Regression (C=1.0)  0.714 $\pm$ 0.064   
     kmeans (eval on recon and supersample)  LRCV                         0.685 $\pm$ 0.038   
                                             Logistic Regression (C=1.0)  0.700 $\pm$ 0.047   
     kmeans (eval on recon)                  LRCV                         0.710 $\pm$ 0.052   
                                             Logistic Regression (C=1.0)  0.710 $\pm$ 0.050   
     kmeans (supersample)                    LRCV                         0.742 $\pm$ 0.043   
                                             Logistic Regression (C=1.0)  0.742 $\pm$ 0.042   
     no_balancing                            LRCV                         0.736 $\pm$ 0.028   
                                             Logistic Regression (C=1.0)  0.735 $\pm$ 0.028   
     ranking                                 LRCV                         0.746 $\pm$ 0.021   
                                             Logistic Regression (C=1.0)  0.747 $\pm$ 0.020   
     ranking (eval on recon and supersample) LRCV                         0.740 $\pm$ 0.002   
                                             Logistic Regression (C=1.0)  0.737 $\pm$ 0.007   
     ranking (eval on recon)                 LRCV                         0.749 $\pm$ 0.007   
                                             Logistic Regression (C=1.0)  0.748 $\pm$ 0.007   
     ranking (supersample)                   LRCV                         0.757 $\pm$ 0.020   
                                             Logistic Regression (C=1.0)  0.757 $\pm$ 0.021   
     true_balancing                          LRCV                         0.744 $\pm$ 0.009   
                                             Logistic Regression (C=1.0)  0.745 $\pm$ 0.008   

                                                                         prob_pos_sex_Male_0.0/sex_Male_1.0  \
type approach                                classifier                                                       
ours kmeans                                  LRCV                                         0.581 $\pm$ 0.330   
                                             Logistic Regression (C=1.0)                  0.579 $\pm$ 0.328   
     kmeans (eval on recon and supersample)  LRCV                                         0.621 $\pm$ 0.104   
                                             Logistic Regression (C=1.0)                  0.562 $\pm$ 0.116   
     kmeans (eval on recon)                  LRCV                                         0.483 $\pm$ 0.205   
                                             Logistic Regression (C=1.0)                  0.482 $\pm$ 0.213   
     kmeans (supersample)                    LRCV                                         0.760 $\pm$ 0.300   
                                             Logistic Regression (C=1.0)                  0.759 $\pm$ 0.297   
     no_balancing                            LRCV                                         0.684 $\pm$ 0.195   
                                             Logistic Regression (C=1.0)                  0.685 $\pm$ 0.197   
     ranking                                 LRCV                                         0.762 $\pm$ 0.119   
                                             Logistic Regression (C=1.0)                  0.763 $\pm$ 0.118   
     ranking (eval on recon and supersample) LRCV                                         0.669 $\pm$ 0.086   
                                             Logistic Regression (C=1.0)                  0.665 $\pm$ 0.077   
     ranking (eval on recon)                 LRCV                                         0.712 $\pm$ 0.088   
                                             Logistic Regression (C=1.0)                  0.712 $\pm$ 0.0

In [14]:
print(m2.to_latex(escape=False))

\begin{tabular}{llllllll}
\toprule
     &                &                             &           Accuracy & prob_pos_sex_Male_0.0/sex_Male_1.0 & TPR_sex_Male_0.0/sex_Male_1.0 & TNR_sex_Male_0.0/sex_Male_1.0 & cluster_context_acc \\
type & approach & classifier &                    &                                    &                               &                               &                     \\
\midrule
baseline & no_balancing & DRO (eta=0.01) &  0.776 $\pm$ 0.028 &                  0.785 $\pm$ 0.122 &              0.78 $\pm$ 0.081 &             0.959 $\pm$ 0.023 &       nan $\pm$ nan \\
     &                & DRO (eta=0.1) &  0.774 $\pm$ 0.029 &                   0.769 $\pm$ 0.13 &             0.765 $\pm$ 0.088 &             0.959 $\pm$ 0.023 &       nan $\pm$ nan \\
     &                & DRO (eta=0.5) &  0.771 $\pm$ 0.028 &                  0.767 $\pm$ 0.139 &              0.757 $\pm$ 0.09 &             0.956 $\pm$ 0.023 &       nan $\pm$ nan \\
     &                &

In [15]:
print(m1.to_latex(escape=False))

\begin{tabular}{llllllll}
\toprule
     &                &                             &           Accuracy & prob_pos_sex_Male_0.0/sex_Male_1.0 & TPR_sex_Male_0.0/sex_Male_1.0 & TNR_sex_Male_0.0/sex_Male_1.0 & cluster_context_acc \\
type & approach & classifier &                    &                                    &                               &                               &                     \\
\midrule
E&S & no_balancing & LRCV &  0.676 $\pm$ 0.029 &                  0.525 $\pm$ 0.262 &             0.555 $\pm$ 0.269 &             0.943 $\pm$ 0.032 &       0.0 $\pm$ 0.0 \\
     &                & Logistic Regression (C=1.0) &  0.677 $\pm$ 0.028 &                   0.525 $\pm$ 0.26 &             0.554 $\pm$ 0.267 &             0.943 $\pm$ 0.032 &       0.0 $\pm$ 0.0 \\
     & ranking & LRCV &  0.606 $\pm$ 0.008 &                  0.298 $\pm$ 0.091 &             0.265 $\pm$ 0.083 &             0.966 $\pm$ 0.014 &   0.695 $\pm$ 0.003 \\
     &                & Logistic Regress